In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import joblib

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix , auc, precision_recall_curve, roc_auc_score
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_validate

In [ ]:
x_train = pd.read_csv('../../assets/gold/experimentos_finales/x_train.csv')
y_train = pd.read_csv('../../assets/gold/experimentos_finales/y_train.csv')

In [ ]:
x_validation = pd.read_csv('../../assets/gold/experimentos_finales/x_test.csv')
y_validation = pd.read_csv('../../assets/gold/experimentos_finales/y_test.csv')

In [ ]:
print(x_train.dtypes.to_markdown())

## Arbol "pelado"

In [ ]:
def auroc_complejidad(model, x, y):
    result = cross_validate(model, x, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=41),scoring='roc_auc',return_train_score=True)
    result_train = np.mean(result["train_score"])
    result_test = np.mean(result["test_score"])

    return result_train, result_test

In [ ]:
x = pd.concat([x_train, x_validation])
y = pd.concat([y_train, y_validation])

In [ ]:
max_depth_values = range(1, 30)

# Inicializamos listas para almacenar las puntuaciones de entrenamiento y prueba
train_auroc = []
test_auroc = []

# Calculamos puntuaciones para cada valor de profundidad máxima
for max_depth in max_depth_values:
    modelo = DecisionTreeClassifier(
                                    criterion='gini',             # Gini
                                    splitter='best',              # Selecciona la mejor división
                                    max_depth=max_depth,          # Sin límite de profundidad
                                    min_samples_split=2,          # Mínimo de muestras para dividir un nodo
                                    min_samples_leaf=1,           # Mínimo de muestras en una hoja
                                    max_features=None,            # Considera todas las características
                                    random_state=42               # Para reproducibilidad
                                )
    auroc_promedio_train, auroc_promedio_test = auroc_complejidad(modelo, x, y)
    train_auroc.append(auroc_promedio_train)
    test_auroc.append(auroc_promedio_test)


# Graficamos las curvas de complejidad
plt.figure(figsize=(10, 6))
plt.plot(max_depth_values, train_auroc, label='Train', marker='o')
plt.plot(max_depth_values, test_auroc, label='Test', marker='o')
plt.xticks(range(1, 30, 2), fontsize=11)
plt.yticks(fontsize=11)
plt.xlabel('Profundidad máxima del árbol', fontsize=13)
plt.ylabel('Área bajo la curva ROC', fontsize=13)
plt.title('Curva de complejidad para árbol de decisión', fontsize=13)
plt.legend(fontsize=13)
plt.show()